In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load a single txt file as a numpy array
def load_file(filepath):
    df = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return df.values

In [3]:
sub_map = load_file("~/Desktop/UCI HAR Dataset/train/subject_train.txt")
trainX = load_file("~/Desktop/UCI HAR Dataset/train/X_train.txt")
trainy = load_file("~/Desktop/UCI HAR Dataset/train/y_train.txt")
print(sub_map.shape)
print(trainX.shape)
print(trainy.shape)

(7352, 1)
(7352, 561)
(7352, 1)


In [9]:
print(type(trainy))
print(trainy.ravel().shape)

<class 'numpy.ndarray'>
(7352,)


In [12]:
from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import f1_score, precision_score

lr = LogisticRegression(solver='lbfgs', max_iter=10000, multi_class='auto')
lr.fit(trainX, trainy.ravel())
# print(lr.coef_)
# print(lr.intercept_)
# print(y)
# print(lr.predict(X))

print("\nOriginal labels ", trainy)
print("Predicted labels", lr.predict(trainX))
print(lr.predict_proba(trainX))

print("Prediction of label for one point: ", lr.predict(trainX))


Original labels  [[5]
 [5]
 [5]
 ...
 [2]
 [2]
 [2]]
Predicted labels [5 5 5 ... 2 2 2]
[[7.45608223e-10 2.06423315e-09 2.68827233e-10 1.08172850e-04
  9.99890400e-01 1.42392225e-06]
 [1.61771573e-07 2.37600371e-09 6.24693618e-09 6.04076202e-03
  9.93947396e-01 1.16717995e-05]
 [6.94976388e-08 2.03039684e-10 5.18881872e-10 1.67691618e-04
  9.99831229e-01 1.00905244e-06]
 ...
 [2.92456279e-03 9.97014776e-01 5.02953849e-05 1.42320975e-06
  1.35661694e-07 8.80706646e-06]
 [5.50167508e-04 9.99438685e-01 6.01471437e-06 8.77963868e-07
  5.56771962e-08 4.19926411e-06]
 [5.37639826e-04 9.99294558e-01 1.61051106e-04 9.87126308e-07
  3.46147102e-07 5.41743176e-06]]
Prediction of label for one point:  [5 5 5 ... 2 2 2]


In [13]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors = 7)
clf.fit(trainX, trainy.ravel())
# print(trainy)
pred_y = clf.predict(trainX)
# print(pred_y)

In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(trainy.ravel(), pred_y)

array([[1226,    0,    0,    0,    0,    0],
       [   0, 1073,    0,    0,    0,    0],
       [   1,    4,  981,    0,    0,    0],
       [   0,    1,    0, 1193,   88,    4],
       [   0,    0,    0,   35, 1339,    0],
       [   0,    0,    0,    1,    0, 1406]])

In [15]:
from sklearn.metrics import classification_report

print(classification_report(trainy.ravel(), pred_y))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      1226
           2       1.00      1.00      1.00      1073
           3       1.00      0.99      1.00       986
           4       0.97      0.93      0.95      1286
           5       0.94      0.97      0.96      1374
           6       1.00      1.00      1.00      1407

    accuracy                           0.98      7352
   macro avg       0.98      0.98      0.98      7352
weighted avg       0.98      0.98      0.98      7352

